In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os,sys,time
from glob import glob
from scipy.optimize import fmin
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
from scipy import interpolate
import sympy as sp
from scipy.fftpack import fft,ifft
import seaborn
plt.rc('font',size=15) 

In [2]:
basebindir = '/Users/laote/sdsu/Research/binary/'
basedir = '/Users/laote/sdsu/Research/binary/curve_bin/'
base3wdir = basedir + '3w_binary/'
baseckdir = basedir + 'Binary_ck/'

In [3]:
def is_in_notebook():
    import sys
    return 'ipykernel' in sys.modules

def clear_output():
    """
    clear output for both jupyter notebook and the console
    """
    import os
    os.system('cls' if os.name == 'nt' else 'clear')
    if is_in_notebook():
        from IPython.display import clear_output as clear
        clear()
        
print('bbb')
clear_output()

# Make light curve

In [4]:
def check_lc(datadir,limx=(0,0),limy=(0,0),error=False):
    data = np.genfromtxt(datadir,names="time,flux,et,ef")
    use = (data['ef'] == 0.)
    mom_dump = data['time'][data['ef'] == 1.64e+02]

    if error == True:
        time_diff = data['time'][use][1:] - data['time'][use][:-1]
        error_point = np.argmax(time_diff)
        Time = np.append(data['time'][use][10:error_point-10],data['time'][use][error_point+10:-10])
        flux = np.append(data['flux'][use][10:error_point-10],data['flux'][use][error_point+10:-10])
    else:
        Time = data['time'][use]
        flux = data['flux'][use]

    med = np.median(flux) 
    std = np.std(flux)
    w = (flux < (med - 3*std))

    plt.figure(figsize=(15,5))
    plt.subplot(121)
    plt.plot(Time,flux,'c')
    plt.plot(Time,flux,'r.')
    plt.axhline(med-3*std,color='g')
    for i in range(len(mom_dump)):
        plt.axvline(mom_dump[i],color='g',ls='--',alpha=0.3)
    plt.xlabel('time');plt.ylabel('flux')
    if limx[0] != 0:
        plt.xlim(limx[0],limx[1])
    plt.grid()
    
    plt.subplot(122)
    plt.plot(Time,flux/med,'c')
    plt.plot(Time,flux/med,'r.')
    plt.axhline(1-3*std/med,color='g')
    for i in range(len(mom_dump)):
        plt.axvline(mom_dump[i],color='g',ls='--',alpha=0.5)
    plt.xlabel('Time');plt.ylabel('$Flux/Flux_{med}$')
    if limx[0] != 0:
        plt.xlim(limx[0],limx[1])
    if limy[0] != 0:
        plt.ylim(limy[0],limy[1])
    plt.grid()
    plt.show()
    
    return len(flux[w])

In [5]:
# norm = 0 let med to 0, norm = 0 let flux divide medium
# D_all = True let all data used
def light_curve(lc_dir,norm=1,D_all=False,look=False):
    data = np.genfromtxt(lc_dir,names="time,flux,et,ef")
    good = (data['ef'] == 0)
    mom_dump = data['time'][data['ef'] == 1.64e+02]
    
    if D_all == True:
        Time = data['time'][good]
        flux = data['flux'][good]
        et = data['et'][good]
    else:
        time_diff = data['time'][good][1:] - data['time'][good][:-1]
        if len(time_diff) < 300:
            return "bad_data","bad_data","bad_data","bad_data"
        if (np.max(time_diff) > 3):
            return "bad_data","bad_data","bad_data","bad_data"
        error_point = np.argmax(time_diff)

        Time = np.append(data['time'][good][5:error_point-10],data['time'][good][error_point+10:-5])
        flux = np.append(data['flux'][good][5:error_point-10],data['flux'][good][error_point+10:-5])
        et = np.append(data['et'][good][5:error_point-10],data['et'][good][error_point+10:-5])

#   norm ot 0 
    if norm == 0:
#         min_flux = np.min(np.append(flux[int(len(flux)*0.05):int(len(flux)*0.4)],\
#                                     flux[int(len(flux)*0.6):-int(len(flux)*0.05)]))
        min_flux = np.min(flux)
        norm_flux = (flux - np.median(flux))/(np.median(flux) - min_flux)
        norm_et = et / np.abs(np.median(flux) - min_flux)
        w = (norm_flux < 0.25) & (norm_flux > -1.05)
#   norm to 1
    if norm == 1:
        norm_flux = flux/np.median(flux)
        norm_et = et/np.median(flux)
#         w = (norm_flux < 1.02) & (norm_flux > 0.2)
        w = (norm_flux < 1.25) & (norm_flux > 0.2)
#   norm to std
    if norm == 2:
        norm_flux = (flux - np.median(flux))/np.std(flux)
        norm_et = et/np.std(flux)
        w = (norm_flux < 15) & (norm_flux > -15)
#   norm to max - min
    if norm == 3:
#         norm_flux = (flux - np.median(flux))/(np.max(flux) - np.min(flux))
        norm_flux = (flux - np.max(flux))/(np.max(flux) - np.min(flux))
        norm_et = et/(np.max(flux) - np.min(flux))
        w = (norm_flux > -2)
    if norm == 0:
        norm_flux = flux
        norm_et = et
        w = (norm_flux > 0)

    if look == True:
        plt.figure(figsize=(10,5))
        for i in range(len(mom_dump)):
            plt.axvline(mom_dump[i],color='g',ls='--',alpha=0.3,linewidth=4)
        plt.plot(Time,norm_flux,'c')
        plt.plot(Time,norm_flux,'r.')
        plt.xlabel('Time (BJD-2455000)');plt.ylabel('Flux (electron/sec)')
        plt.grid()
        plt.show()
    
    return Time[w],norm_flux[w],norm_et[w],mom_dump

def mulit_light_curve(lc_dir,limx=(0,0),limy=(0,0),norm=1,D_all=False,look=False):
    Time = []; flux = []; et = []; mom_dump = []
    size = 0; sec_size = []
    for i in range(20):
        test_dir = lc_dir[:-10] + '0%.2i'%i + "_LC.txt"
        if os.path.exists(test_dir) == True:
            t,f,e,md = light_curve(test_dir,norm,D_all)
            if t != 'bad_data':
                Time.extend(t)
                flux.extend(f)
                et.extend(e)
                mom_dump.extend(md)
                size += 1
                sec_size.append(i)
            
    if look == True:
        plt.figure(figsize=(5*(2+1),5))
#         plt.title('SEC size = %s'%sec_size)
        for i in range(len(mom_dump)):
            plt.axvline(mom_dump[i],color='g',ls='--',alpha=0.3,linewidth=4)
        plt.plot(Time,flux,'c')
        plt.plot(Time,flux,'r.')
        plt.xlabel('Time (BJD-2455000)');plt.ylabel('Normlized Flux')
        if limx[0] != 0:
            plt.xlim(limx[0],limx[1])
        if limy[0] != 0:
            plt.ylim(limy[0],limy[1])
        plt.grid()
        plt.show()

    return Time, flux, et, mom_dump

# Make Figure for light curve

In [6]:
def Figure_create(datadir,datadir2,sv=False):
    data = np.genfromtxt(datadir,names="time,flux,et,ef")
    good = (data['ef'] == 0)
    
    Time = data['time'][good]
    flux = data['flux'][good]
    et = data['et'][good]

    data2 = np.genfromtxt(datadir2,names="time,flux,et,ef")
    good = (data2['ef'] == 0)
    
    Time2 = data2['time'][good]
    flux2 = data2['flux'][good]
    et2 = data2['et'][good]
    
    
    plt.figure(figsize=(15,5))
    plt.subplot(121)
    plt.plot(Time,flux,'c',label='%s'%(datadir[-28:-14]))
    plt.plot(Time,flux,'r.',alpha=0.5)
    plt.xlabel('BJD-2455000');plt.ylabel('Original Flux')
    plt.grid()
    plt.legend(loc='upper left')
    
    plt.subplot(122)
    plt.plot(Time2,flux2,'c',label='%s'%(datadir2[-28:-14]))
    plt.plot(Time2,flux2,'r.',alpha=0.5)
    plt.xlabel('BJD-2455000');plt.ylabel('Original Flux')
    plt.grid()
    plt.legend(loc='upper left')
    
#     t,f,e = light_curve(datadir,norm=0,D_all=True)
#     plt.subplot(222)
#     plt.plot(t,f,'c')
#     plt.plot(t,f,'r.',alpha=0.5)
#     plt.xlabel('BJD-2455000');plt.ylabel('Normalized Flux')
#     plt.grid()
    
#     t,f,e = light_curve(datadir,norm=1,D_all=True)
#     plt.subplot(223)
#     plt.plot(t,f,'c')
#     plt.plot(t,f,'r.',alpha=0.5)
#     plt.xlabel('BJD-2455000');plt.ylabel('Normalized Flux')
#     plt.grid()

#     t,f,e = light_curve(datadir,norm=2,D_all=True)
#     plt.subplot(224)
#     plt.plot(t,f,'c')
#     plt.plot(t,f,'r.',alpha=0.5)
#     plt.xlabel('BJD-2455000');plt.ylabel('Normalized Flux')
#     plt.grid()
    if sv == True:
        plt.savefig('/Users/laote/sdsu/Research/binary/Figure/%s.png'%(datadir[-28:-14]))
    plt.show()

# Find Eclipse time by fitpeaks

In [7]:
def fit_peak(lc_dir,h=3,look=False):
    Time, flux, et, _ = mulit_light_curve(lc_dir,norm=2)
    Time = np.array(Time); flux = np.array(flux); et = np.array(et)
    
    peaks = find_peaks(-flux, height=2, distance=10)
    
    if len(peaks[0]) < int(len(Time)/60):
        peaks = find_peaks(-flux, height=h, distance=10)

    peaks_pos = peaks[0]
    height = -peaks[1]['peak_heights']
    
    need_del = []
    for i in range(len(Time[peaks_pos])):
        if ((peaks_pos + 5)[i] > len(Time)) or ((peaks_pos - 5)[i] < 0) :
            need_del.append(i)
            
    peaks_pos = np.delete(peaks_pos, need_del)
    height = np.delete(height, need_del)
    fittime = Time[peaks_pos]

    if look == True:
        plt.figure(figsize=(10,5))
#         plt.subplot(121)
        plt.plot(Time,flux,'c.-')
        plt.plot(fittime,height,'rx')
        plt.xlabel('Time');plt.ylabel('Flux')
        plt.grid()

#         plt.subplot(122)
#         plt.plot(Time,flux,'co-')
#         plt.plot(fittime,height,'rx')
#         if len(fittime) > 2:
#             plt.xlim(fittime[0]-0.2,fittime[2]+0.2)
#         plt.grid()
        plt.show()
    
    return np.array(fittime),np.array(height)

# Folding Light Curve

In [8]:
def Phase_cycle(t,P,T_0):
    T_diff = np.array(t) - T_0
    I = np.round(T_diff/P)
    phi = (T_diff/P) - I
    return phi, I #phase and cycle


def phase_average(x,y,bins=100):
    if bins >= len(x):
        return np.array(x),np.array(y)
    else:
        size = int(len(x)/bins)
        newx = []; newy = []
        for i in range(bins):
            newx.append(np.mean(x[i*size:(i+1)*size]))
            newy.append(np.mean(y[i*size:(i+1)*size]))
        newx.append(np.mean(x[bins*size:]))
        newy.append(np.mean(y[bins*size:]))
        return np.array(newx),np.array(newy)
    

def sing_dc_func(lc_dir,guess_P,look=False):
    ecl_time, height = fit_peak(lc_dir,h=3.5)
    ecl_phi, ecl_I = Phase_cycle(ecl_time,guess_P,ecl_time[0])
    ecl_flux = np.stack((ecl_phi,height),axis=-1)
    ecl_flux = np.array(sorted(ecl_flux,key=lambda x:x[0]))
    
    Time, flux, et, _ = mulit_light_curve(lc_dir,norm=2)
    if len(Time) < 1:
        return 0,0,0
    Time = np.array(Time); flux = np.array(flux); et = np.array(et)
    
    phi, I = Phase_cycle(Time, guess_P, ecl_time[0])
    phase_flux = np.stack((phi,flux),axis=-1)
    phase_flux = np.array(sorted(phase_flux,key=lambda x:x[0]))
    
    phase = phase_flux[:,0]
    flux = phase_flux[:,1]

    mphase,mflux = phase_average(phase,flux,int(len(phase)/10))
    
    if look == True:
        plt.figure(figsize=(10,5))
#         plt.title('%s -- period = %.6f'%(lc_dir[-28:-14],guess_P))
        plt.plot(phase,flux,'c-',alpha=0.5)
        plt.plot(phase,flux,'r.',alpha=0.5)
#         plt.plot(mphase,mflux,'k-')
#         bbs = np.linspace(-0.5,0.5,50)
#         print(bbs)
#         for ss in bbs:
#             plt.axvline(ss,ls='--',color='b')
#         plt.plot(ecl_flux[:,0],ecl_flux[:,1],'ko')
        plt.xlabel('Phase');plt.ylabel('Normlized Flux (STD)')
        plt.grid()
#         plt.savefig('./img/fold_1_ tic00007433670.png',bbox_inches = "tight")
        plt.show()        
    
    S = np.fabs(flux[-1] - flux[0]) + np.sum(np.fabs(flux[1:] - flux[:-1]))
    return S/47.-2.,phase,flux

# Make a general guess period
Use histogram test the folding period

In [1]:
def double_period(t,f,period):
    flux_diff = np.abs(f[1:] - f[:-1])
    _,I = Phase_cycle(t[:-1],period,t[0])
    if np.mean(flux_diff[I%2==0]) > 0.7:
        period = period*2
    return period

def guess_period(t,f): # eclipse time and flux
    time_diff = np.sort(t[1:] - t[:-1])
    period_var = np.abs(time_diff[1:]-time_diff[:-1])
    
    #general case
    a = 0
    while ( period_var[a] > 0.07*time_diff[a] ) & ( a < (len(period_var)-1) ):
        a += 1    
    print(a)
    b = a+int(len(period_var)*0.5)+1 #half number of points
    if ( b <= len(period_var) ): 
        fg = period_var[a:b] 
        if np.all(fg < 0.07*time_diff[a]):
            period = time_diff[a]
            period = double_period(t,f,period)
            return period
        
    #special case
    if len(t) == 4:
        if np.abs(f[2]-f[1]) < 0.4:
            period = t[2]-t[1]
            return period
        p1 = t[2]-t[0];p2 = t[3]-t[1]
        if np.abs(p1-p2) < 0.07*p1:
            period = p1
        else:
            period = t[2]-t[1]
        return period

    if len(t) == 5:
        p1 = t[2]-t[0]; p2 = t[3]-t[1]; p3 = t[4]-t[2]
        if (np.abs(p1-p3) < 0.07*p1):
            period = p1
        else:
            period = np.min([p1,p2,p3])
        return period

    
    time_diff_2 = np.sort(t[2:] - t[:-2])
    period_var_2 = np.abs(time_diff_2[1:]-time_diff_2[:-1])
    a = 0
    while ( period_var_2[a] > 0.07*time_diff_2[a] ) & ( a < (len(period_var_2)-1) ):
        a += 1    
    if ( a < (len(period_var_2)-1) ):
        period = time_diff_2[a]
    else:
        period = time_diff_2[0]
    return period


def hist_test(lc_dir,period=0,look=False):
    ecl_time, height = fit_peak(lc_dir,h=3.5)
    if len(ecl_time) < 2:
        return 0,0,0   
    if len(ecl_time) < 4:
        if np.abs(height[-1] - height[0]) > 1:
            return 0,0,0
        real_p = ecl_time[-1] - ecl_time[0]
        ecc_period = 999
        return real_p, ecc_period, ecl_time[0]  
    if period == 0:
        period = guess_period(ecl_time,height)
        
    T0 = ecl_time[0]
    bin_num = 50
    limit = 0.1*period
    ppp = np.linspace(period-limit,period+limit,5000)
    hist_weight = []
    for pp in ppp:
        phi, I = Phase_cycle(ecl_time,pp,T0)
        hist_info = np.histogram(phi,bins=bin_num)[0]
        nonzero = len(hist_info[hist_info != 0])
        hist_weight.append(np.max(hist_info)-nonzero)
        
    real_p = ppp[np.argmax(hist_weight)]
    
    if look == True:
#         plt.figure(figsize = (10,5))
#         plt.subplot(121)
# #         plt.title('period = %.6f' %real_p)
#         plt.plot(ppp,hist_weight,'c')
#         plt.xlabel('period (days)');plt.ylabel('W')
        
#         phi, I = Phase_cycle(ecl_time,real_p,T0)
#         plt.subplot(122)
#         plt.hist(phi,bins=bin_num)
#         plt.xlabel('Phase')
#         plt.show()
        
        fig, axs = plt.subplots(1,4,figsize=(15,4))
        
        axs[0].plot(ppp,hist_weight,'c')
        axs[0].minorticks_on()
        axs[0].tick_params(which='both',top='on',bottom='on',left='on',right='on')
        axs[0].tick_params(which='major',direction='in',length=10)
        axs[0].tick_params(which='minor',direction='in',length=5)
        axs[0].set_xlabel('Period (days)')
        axs[0].set_ylabel('$W_h$')
        axs[0].grid()
        plot_p = [real_p,1.215,4.732]
        for i in range(1,4):
            phi, I = Phase_cycle(ecl_time,plot_p[i-1],T0)
            axs[i].hist(phi,bins=bin_num,label='P = %.3f'%plot_p[i-1])
            axs[i].minorticks_on()
            axs[i].tick_params(which='both',top='on',bottom='on',left='on',right='on')
            axs[i].tick_params(which='major',direction='in',length=10)
            axs[i].tick_params(which='minor',direction='in',length=5)
            axs[i].set_xlabel('Phase Bins')
            axs[i].set_ylabel('Number of Eclipse')
            axs[i].set_yticks([0,2,4,6,8,10])
            axs[i].set_ylim(0,11.5)
            axs[i].legend()
        plt.tight_layout()
#         plt.savefig('./img/ht.png', bbox_inches = "tight")
        plt.show()
    
    phi, I2 = Phase_cycle(ecl_time,real_p,T0)
    sigy = np.sqrt(1/(len(ecl_time)-2)* np.sum((ecl_time-I2*real_p-T0)**2))
    Delta = len(ecl_time)*np.sum(ecl_time**2) - (np.sum(ecl_time))**2
    ecc_period = sigy * np.sqrt(len(ecl_time)/Delta)
    
    return real_p, ecc_period, ecl_time[0]

# Fit each eclipse to get percise period
Need a rough period

In [10]:
def first_poly(x,a,b):
    y = a*x+b
    return y

def sec_poly(x,a,b,c):
    y = a*x**2+b*x+c
    return y

def fourth_poly(x,a,b,c,d,e):
    y = a*x**4+b*x**3+c*x**2+d*x+e
    return y

def sixth_poly(x,a,b,c,d,e,f,g):
    y = a*(x**6) + b*(x**5) + c*(x**4) + d*(x**3) + e*(x**2) + f*x + g
    return y

def root_4th_poly(a,b,c,d):
    x = sp.Symbol('x')
    f = 4*a*(x**3) + 3*b*(x**2) + 2*c*(x) + d
    root = sp.solve(f)
    r = [complex(root[0]).real,complex(root[1]).real,complex(root[2]).real]
    mid = r[np.argmin(np.abs(r))]
    return mid
    

def primary_eci(time,flux,period,eci_1st=0):
    
    # find the minimum flux in first period to be the first primary eclipse
    
    if eci_1st == 0:
        peaks = find_peaks(-flux, height=0.5)
        print(peaks)
        peaks_pos = peaks[0]
        eci_1st = time[peaks_pos][0]
    else:
        None
    
    w = (time > eci_1st-0.75*period) & (time < eci_1st+0.75*period)
    if len(flux[w]) > 0:
        tp = np.argmin(flux[w])
        t = time[w][tp]
        while t > (time[0]-period):
            t -= period
        return t
    else:
        return eci_1st
        
    
def find_eclipse(lc_dir,period,eci_1st=0,look=False):
    time, flux, et, _ = mulit_light_curve(lc_dir,norm=0)
    time = np.array(time); flux = np.array(flux); et = np.array(et) #set list to array
    
    if time.size == 0:
        return [0]
    prim_eci = primary_eci(time,flux,period,eci_1st)
    
    ck_time = np.arange(prim_eci,time[-1],period) #check each period from the begining of first eclipse
    p1_time = []; p1_flux = []; p_time = []; p_flux = []
    for i in range(len(ck_time)):
        tn = ck_time[i]
        tr = ((tn - 0.2*period) < time) & (time < (tn + 0.2*period)) #create a box around eclipse to fit
        if len(time[tr] > 5):
            mid_pos = np.argmin(flux[tr]) #mid eclipse position in time range
            left_pos = 0;right_pos = 0; 
            while (flux[tr][mid_pos-left_pos] < -0.1) & ((left_pos + 2) < mid_pos):
                left_pos += 1
            while (flux[tr][mid_pos+right_pos] < -0.1) & ((right_pos + 2) < (len(flux[tr]) - mid_pos)):
                right_pos += 1
            lp = mid_pos-left_pos #left eclipse position in time range
            rp = mid_pos+right_pos #right eclipse position in time range

            duration = right_pos+left_pos #number of point during eclipse
            if (np.min(flux[tr]) < -0.5) & (left_pos > 1) & (right_pos > 1):
                x = time[tr][lp:rp] - time[tr][mid_pos]
                y = flux[tr][lp:rp]
                testx = np.linspace(x-0.3,x+0.3,100)

                if duration < 5: #  2nd poly fitting
                    popt, pcov = curve_fit(sec_poly, x, y, p0=[100,0,-0.5])

                    if popt[0] <= 0:
                        continue

                    peak_x = -0.5*popt[1]/popt[0]
                    peak_y = sec_poly(peak_x,popt[0],popt[1],popt[2])
                    testy = sec_poly(testx,popt[0],popt[1],popt[2])
                    # uncertainty
                    peak_ex = np.sqrt((1/popt[0])**2*pcov[1,1]**2+(popt[1]/popt[0]**2)**2*pcov[0,0]**2)
                    
                else: #  4th poly fitting
                    popt, pcov = curve_fit(fourth_poly, x, y, p0=[100,0,0,0,-0.9])

#                     if popt[0] >= 0:
#                         continue
                    peak_x = root_4th_poly(popt[0],popt[1],popt[2],popt[3])
                    peak_y = fourth_poly(peak_x,popt[0],popt[1],popt[2],popt[3],popt[4])
                    testy = fourth_poly(testx,popt[0],popt[1],popt[2],popt[3],popt[4])

#################  check each fitting #################
                
#                 plt.plot(testx,testy,'c-')
#                 plt.plot(peak_x,peak_y,'ko')
#                 plt.plot(time[tr]-time[tr][mid_pos],flux[tr],'r.')
#                 plt.ylim(-1.5,0.2)
#                 plt.show()
                
#######################################################
                
    
#################  append time and flux ###############

                p1_time.append(time[tr][np.argmin(flux[tr])]) # local minimum record
                p1_flux.append(np.min(flux[tr]))

                p_time.append(peak_x+time[tr][mid_pos]) # fitting time
                p_flux.append(peak_y)
                
#######################################################
                

#################  check peak position ####################

    if look == True:
        plt.figure(figsize=(25,5))
        plt.plot(time,flux,'c.-')
        plt.plot(p_time,p_flux,'rx')
        plt.plot(p1_time,p1_flux,'b.')
#         plt.xlim(3550,3650)
        plt.ylim(-1.25,0.2)
        plt.xlabel('time')
        plt.ylabel('normlized flux')
        plt.show()
    
###########################################################
    
    return p_time

In [11]:
def check_eci(lc_dir,period,eperiod,eci_1st=0,look=False):
    p_time = np.array(find_eclipse(lc_dir,period,eci_1st,look))
    if len(p_time) > 3:
        phi,I = Phase_cycle(p_time,period,p_time[0])
    
        popt, pcov = curve_fit(first_poly, I, p_time)
        testy = first_poly(I,popt[0],popt[1])
        res = p_time-testy
        
        med = np.median(res)
        std = np.std(res)
        w = (res > (med - 3*std)) & (res < (med + 3*std)) & (res > -0.05) & (res < 0.05)
        
        if len(p_time[w]) > 4:
            popt, pcov = curve_fit(first_poly, I[w], p_time[w])
            testy = first_poly(I[w],popt[0],popt[1])
            res = p_time[w]-testy

            if look == True:
                plt.figure(figsize=(10,4))
                plt.plot(I[w],res,'r.-')
                plt.title('period = %s'%popt[0])
                plt.ylabel('phase')
                plt.xlabel('cycle')
                plt.show()
            
        if len(p_time[w]) > 0:
            eci_1st = p_time[w][0]
        else:
            eci_1st = p_time[0]

        return p_time[w],popt[0],np.sqrt(pcov[0,0]),eci_1st
    else:
        return None,period,eperiod,eci_1st

# Fit single eclipse for special case

In [12]:
def shift_func(x,shift):
    a,b,c,d,e = [-1.41291749e+03, -2.22694480e+00,
             3.62497542e+01,  5.67484897e-03,
             7.30504346e-01]
    y = fourth_poly(x+7.827378247115863e-05-shift,a,b,c,d,e)
    return y

def shift_func(x,shift):
    a,b,c,d,e = [-4.39190648e+02,  3.14476753e+00,  
                 1.34154599e+01, -2.91334914e-02, 8.89616078e-01]
    y = fourth_poly(x-0.0010854872769290647-shift,a,b,c,d,e)
    return y

def find_eclipse_single(lc_dir,period,eci_pos,look=False): #eci_pos is an array of each ecilpse time
    time, flux, et, _ = mulit_light_curve(lc_dir,norm=1,D_all=True)
    time = np.array(time); flux = np.array(flux); et = np.array(et) #set list to array
    ck_time = np.array(eci_pos)
    p_time = []; p_flux = []
    
    size = int(np.sqrt(len(ck_time)))+1
    plt.figure(figsize=(3*size,3*size))
    for i in range(len(ck_time)):
        tn = ck_time[i]
        tr = ((tn - 0.08*period) < time) & (time < (tn + 0.08*period)) #create a box around eclipse to fit
        if len(time[tr] > 5):
            mid_pos = np.argmin(flux[tr]) #mid eclipse position in time range
            left_pos = 0;right_pos = 0; 
            while (flux[tr][mid_pos-left_pos] < 0.999) & ((left_pos + 2) < mid_pos):
                left_pos += 1
            while (flux[tr][mid_pos+right_pos] < 0.999) & ((right_pos + 2) < (len(flux[tr]) - mid_pos)):
                right_pos += 1
            lp = mid_pos-left_pos #left eclipse position in time range
            rp = mid_pos+right_pos #right eclipse position in time range

            duration = right_pos+left_pos #number of point during eclipse
            if (left_pos > 1) & (right_pos > 1):
                x = time[tr][lp-1:rp+1] - time[tr][mid_pos]
                y = flux[tr][lp-1:rp+1]
                testx = np.linspace(x-0.05,x+0.05,100)
                if duration < 5: #  2nd poly fitting
                    popt, pcov = curve_fit(sec_poly, x, y, p0=[100,0,-0.5])
                    if popt[0] <= 0:
                        continue
                    peak_x = -0.5*popt[1]/popt[0]
                    peak_y = sec_poly(peak_x,popt[0],popt[1],popt[2])
                    testy = sec_poly(testx,popt[0],popt[1],popt[2])
                    # uncertainty
                    peak_ex = np.sqrt((1/popt[0])**2*pcov[1,1]**2+(popt[1]/popt[0]**2)**2*pcov[0,0]**2)
                    
#                 if duration < 50: #  shift fit
#                     popt, pcov = curve_fit(shift_func, x, y, p0=[0])
#                     peak_x = popt[0]
#                     peak_y = shift_func(peak_x,popt[0])
#                     testy = shift_func(testx,popt[0])
#                     # uncertainty
#                     peak_ex = pcov[0]
#                     print(peak_ex)
                    
                else: #  4th poly fitting
                    popt, pcov = curve_fit(fourth_poly, x, y, p0=[1,1,1,1,1])
                    peak_x = root_4th_poly(popt[0],popt[1],popt[2],popt[3])
                    peak_y = fourth_poly(peak_x,popt[0],popt[1],popt[2],popt[3],popt[4])
                    testy = fourth_poly(testx,popt[0],popt[1],popt[2],popt[3],popt[4])  
                      #  6th poly fitting
#                     popt, pcov = curve_fit(sixth_poly, x, y, p0=[1,1,1,1,1,1,1])
#                     peak_x = 0
#                     peak_y = flux[tr][mid_pos]
#                     testy = sixth_poly(testx,popt[0],popt[1],popt[2],popt[3],popt[4],popt[5],popt[6])
                    

                p_time.append(peak_x+time[tr][mid_pos]) # fitting time record
                p_flux.append(peak_y)
               
                plt.subplot(size,size,i+1)
                plt.plot(testx+time[tr][mid_pos],testy,'c-')
                plt.plot(peak_x+time[tr][mid_pos],peak_y,'ko')
                plt.plot(time[tr],flux[tr],'r.')
                plt.ylim(peak_y-0.05,1.01)
                plt.grid()
                plt.tight_layout()
    plt.show()
                
#################  check peak position ####################

    if look == True:
        plt.figure(figsize=(25,5))
        plt.plot(time,flux,'c.-')
        plt.plot(p_time,p_flux,'rx')
        plt.xlabel('time')
        plt.ylabel('normlized flux')
#         plt.ylim(0.91,1.01)
        plt.show()
    
###########################################################
    
    return p_time

# Fourier Transform

In [13]:
def fourier(lc_dir,look=False):
    x,y,z,_ = mulit_light_curve(lc_dir,norm=2)
    if len(y) == 0:
        return 0
    #采样点选择1400个，因为设置的信号频率分量最高为600赫兹，根据采样定理知采样频率要大于信号频率2倍，所以这里设置采样频率为1400赫兹（即一秒内有1400个采样点，一样意思的）
    # x=np.linspace(0,1,1400)      

    #设置需要采样的信号，频率分量有180，390和600
    # y=7*np.sin(2*np.pi*10*x)

    yy=fft(y)                     #快速傅里叶变换
    yreal = yy.real               # 获取实数部分
    yimag = yy.imag               # 获取虚数部分

    yf=abs(fft(y))                # 取绝对值
    yf1=abs(fft(y))/len(x)           #归一化处理
    yf2 = yf1[range(int(len(x)/2))]  #由于对称性，只取一半区间

    xf = np.arange(len(y))        # 频率
    xf = xf/(x[-1]-x[0])
    xf1 = xf
    xf2 = xf[range(int(len(x)/2))]  #取一半区间

    if look == True:
        plt.figure(figsize=(15,4))
        plt.subplot(121)
        plt.plot(x,y,'b')   
        plt.xlabel('Time (BJD-2455000)')
        plt.ylabel('Normlized Flux (STD)')
#         plt.title('Original wave')

#         plt.subplot(222)
#         plt.plot(xf,yf,'r')
#         plt.title('FFT of Mixed wave(two sides frequency range)',fontsize=7,color='#7A378B')  #注意这里的颜色可以查询颜色代码表

#         plt.subplot(223)
#         plt.plot(xf1,yf1,'g')
#         plt.title('FFT of Mixed wave(normalization)',fontsize=9,color='r')

        plt.subplot(122)
        plt.plot(xf2,yf2,'b')
        plt.xlabel('Frequency (1/days)')
        plt.ylabel('Amplitude')
#         plt.title('FFT of Mixed wave)',fontsize=10,color='#F08080')
#         plt.xlim(-0.5,5)
        plt.tight_layout()
        plt.savefig('./img/FFT_%s.png'%lc_dir[-28:-14],bbox_inches = "tight")
#

        plt.show()
    
    if np.max(yf2[int(0.05*len(yf1)):]) > 0.1:
        return 2 #sine curve
    if np.max(yf2[int(0.05*len(yf1)):]) > 0.05:
        return 1 #eclipse curve or weak sine
    else:
        return 0 #noise
  

# Automatically find period (Failed)

In [14]:
def auto_period(lc_dir,look=False):
    Time, flux, et, _ = mulit_light_curve(lc_dir)
    Time = np.array(Time);flux = np.array(flux)
    func = interpolate.interp1d(Time,flux)
    t_new = np.linspace(Time[0],Time[-1],20*len(Time))
    f_new = func(t_new)
    
    diff = []
    for i in range(1,int(0.25*len(f_new))):
        diff.append(np.mean(np.abs(f_new[i:2*i]-f_new[2*i:3*i])))
    diff = np.array(diff)
    
    peaks = find_peaks(-diff, height=-0.05, width=50)

    peaks_pos = peaks[0]
    w = (peaks_pos > np.argmax(diff[:int(0.1*len(f_new))]))
    peaks_pos = peaks_pos[w]
    
    if len(peaks_pos) == 0:
        return 0
    
    if len(peaks_pos) > 2:
        total = peaks_pos[-1]//peaks_pos[0]
        period = (Time[-1]-Time[0])/len(f_new) * peaks_pos[-1]/total
    else:
        period = (Time[-1]-Time[0])/len(f_new) * peaks_pos[0]
    
    if look == True:
        plt.figure(figsize=(15,5))
        plt.title('period = %.6f'%period)
        plt.plot(diff,'c')
        for pos in peaks_pos:
            plt.axvline(x=pos,color='r')
        plt.ylabel('diff')
        plt.grid()
        plt.show()

    return period

# Make a period 'txt' file

In [15]:
def output_info(data,filename): #data is np.array
    a = data
    f = open('%s.txt'%filename,'w')
    f.write('tic_name           period      e_period      first_eclipse\n')
    for i in range(len(a)):
        if np.float(a[i,1]) != 0:
            f.write('%s  %11.6f  %11.6f  %13.6f\n'%(a[i,0],np.float(a[i,1]),np.float(a[i,2]),np.float(a[i,3])))
    f.close()
    
# for key in eci_time:
#     if np.array(eci_time[key]).dtype == 'float64':
#         f = open('./curve_bin/Binary_pe/%s.txt'%(key),'w')
#         for ele in eci_time[key]: 
#             f.write('%.5f\n'%(ele))
#         f.close()

In [16]:
def eye_check(checkdir,limx=[0,0],limy=[0,0],blist=(basedir+'binary_list_928.txt')):
    p = get_known_period(checkdir[-28:-14],blist)
    check_lc(checkdir,limx,limy)
    _ = mulit_light_curve(checkdir,limx,limy,look=True)
    _ = sing_dc_func(checkdir,p,True)   

# Sort Light Curve list by STD

In [17]:
def std_sort(lc_dir_list):
    std = [];new_list = []
    for lc_dir in lc_dir_list:
        Time, flux, et, _ = mulit_light_curve(lc_dir)
        Time = np.array(Time);flux = np.array(flux)
        p = get_known_period(lc_dir[-28:-14],(basedir+'binary_list.txt'))
        if len(flux) < 30:
            continue
        if (p > 30) | (p < 4) | (np.sort(flux)[20] > 0.9):
            continue
        med = np.median(flux)
        sd = np.std(flux)
        w = (flux > med - 2*sd) & (flux < med + 2*sd)
        std.append(np.std(flux[w]))
        new_list.append(lc_dir)
    std = np.array(std)
    new_list = np.array(new_list)
    return new_list[np.argsort(std)]

# Check nearby star

In [ ]:
def near_star(lc_dir,limy=[0,0],rg=300):
    number = np.int(lc_dir[-25:-14])
    length = len(str(number))
    plt.figure(figsize=(10,5))
    for i in range(number-rg,number+rg):
        test_dir = lc_dir[:(-14-length)] + "%s" %i + lc_dir[-14:]
        if os.path.exists(test_dir) == True:
            t,f,e = light_curve(test_dir,norm=1,D_all=True)  
            if test_dir == lc_dir:
                plt.plot(t,f,'r-',label='%s'%i)
            else: 
                plt.plot(t,f,'--.',alpha=0.3,label='%s'%i)
    plt.xlabel('BJD-2455000');plt.ylabel('flux')
    if limy[0] != 0:
        plt.ylim(limy[0],limy[1])
    plt.grid()
    plt.legend()
    plt.show() 

# Get known (or unknown) binary tic name

In [ ]:
def get_known_binary(datadir,blist,good=False):
    binary_name_all = glob(datadir)
    known_binary = np.genfromtxt(blist,dtype=str,skip_header=1)
    known_tic_name = known_binary[:,0]
    known_tic_p = known_binary[:,1].astype(float)
    known_tic_ep = known_binary[:,2].astype(float)
    w = (known_tic_ep > 0) & (known_tic_ep < 1) & (known_tic_p > 0)
    if good == True:
        known_tic_name = known_tic_name[w]

    tname = [];tdir = [];ccc = [];altdir = []
    for a in binary_name_all:
        if a[-28:-14] in known_tic_name:
            if a[-28:-14] not in ccc:
                tname.append(a[-28:-14])
                tdir.append(a)
            ccc.append(a[-28:-14])
            altdir.append(a)
    tname = sorted(tname, key=lambda name: int(name[3:]))
    tdir = sorted(tdir, key=lambda name: int(name[-25:-14] + name[-10:-7]))
    return tname,tdir,altdir

In [ ]:
def get_unknown_binary(datadir,blist,good=False):
    binary_name_all = glob(datadir)
    known_binary = np.genfromtxt(blist,dtype=str,skip_header=1)
    known_tic_name = known_binary[:,0]
    known_tic_p = known_binary[:,1].astype(float)
    known_tic_ep = known_binary[:,2].astype(float)
    w = (known_tic_ep > 0) & (known_tic_ep < 1) & (known_tic_p > 0)
    if good == True:
        known_tic_name = known_tic_name[w]

    tname = [];tdir = [];ccc = [];altdir = []
    for a in binary_name_all:
        if a[-28:-14] not in known_tic_name:
            if a[-28:-14] not in ccc:
                tname.append(a[-28:-14])
                tdir.append(a)
            ccc.append(a[-28:-14])
            altdir.append(a)
    tname = sorted(tname, key=lambda name: int(name[-25:-14] + name[-10:-7]))
    return tname,tdir,altdir

In [ ]:
def get_known_period(ticname,blist):
    known_binary = np.genfromtxt(blist,dtype=str,skip_header=1)
    known_tic_name = known_binary[:,0]
    known_tic_p = known_binary[:,1].astype(float)
    ticp = known_tic_p[np.argwhere(known_tic_name == ticname)]
    return ticp[0][0]

# Get momenutm dump time

In [1]:
def get_mom_dump(lc_list,write=False):
    j = 1
    datadir = []
    for i in range(len(lc_list)):
        if (lc_list[i][-10:-7] == ('0%.2i'%j)):
            j+=1
            datadir.append(tdir[i])

    mom_dump = []
    for dd in datadir:
        data = np.genfromtxt(dd,names="time,flux,et,ef")
        for i in range(len(data)):
            if data['ef'][i] == 1.640000000000000000e+02:
                mom_dump.append(data['time'][i])

    if write == True:
        f = open('mom_dump.txt','w')
        for i in range(len(mom_dump)):
            f.write('%s\n'%(mom_dump[i]))
        f.close()

    return mom_dump